In [23]:
%cd ~/change
import pickle
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

/home/sibava/change


In [24]:
tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model = AutoModelForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
with open('data/mclose.pickle',mode='rb') as f:
    train_x = pickle.load(f)
    train_y = pickle.load(f)

In [49]:
with open('mclose_fugashi.pickle',mode='rb') as f:
    train_x = pickle.load(f)
    train_y = pickle.load(f)

In [26]:
input_ids = tokenizer.encode(f'願望{tokenizer.mask_token}込める',return_tensors='pt')
masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
result = model(input_ids)
pred_ids = result.logits[:, masked_index].topk(5).indices.tolist()[0]
for pred_id in pred_ids:
    output_ids = input_ids.tolist()[0]
    output_ids[masked_index] = pred_id
    print(tokenizer.decode(output_ids))

[CLS] 願望 を 込める [SEP]
[CLS] 願望 に 込める [SEP]
[CLS] 願望 も 込める [SEP]
[CLS] 願望 の 込める [SEP]
[CLS] 願望 が 込める [SEP]


In [21]:
result.logits

tensor([[[-6.3272,  6.8766, -3.6900,  ..., -6.9439, -6.4129, -6.8763],
         [-4.5445,  7.6031, -4.2771,  ..., -5.6963, -4.9951, -5.5726],
         [-5.8127,  7.1981, -4.5688,  ..., -7.7368, -5.9086, -3.5390],
         [-6.6963,  6.4869, -4.4154,  ..., -5.1872, -4.4481, -1.4142],
         [-6.0268,  7.7390, -7.0289,  ..., -5.8504, -6.1021, -9.0389],
         [-9.2925,  6.1579, -5.2319,  ..., -4.0111, -7.6864, -9.0090]]],
       grad_fn=<AddBackward0>)

In [27]:
kaku_dict ={
    'から':['から','が'],
    'が':['を'],
    'で':['が','で'],
    'と':['と'],
    'に':['が','に'],
    'の':['を'],
    'へ':['へ'],
    'まで':['まで'],
    'を':['を']
    }

kaku_dict_before ={
    'から':['から','が'],
    'が':['が','に','を','の'],
    'で':['が','で'],
    'と':['と','に'],
    'に':['が','で','に','を'],
    'の':['を'],
    'へ':['へ'],
    'まで':['まで'],
    'を':['を']
    }

In [28]:
kakuid_dict = {}
for k,v in kaku_dict.items():
    kakuid_dict[tokenizer.encode(k,add_special_tokens=False)[0]] = tokenizer.encode(v,add_special_tokens=False)
    print(k,v)
print(kakuid_dict)

から ['から', 'が']
が ['を']
で ['が', 'で']
と ['と']
に ['が', 'に']
の ['を']
へ ['へ']
まで ['まで']
を ['を']
{40: [40, 14], 14: [11], 12: [14, 12], 13: [13], 7: [14, 7], 5: [11], 118: [118], 126: [126], 11: [11]}


In [ ]:
counter = 0
for i in range(len(train_x)):
    input_ids = tokenizer.encode(f'{train_x[i][1]}{tokenizer.mask_token}{train_x[i][2]}',return_tensors='pt')
    masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    result = model(input_ids)
    pred_ids = result.logits[:, masked_index].topk(10).indices.tolist()[0]
    token_list = tokenizer.decode(pred_ids)
    candidate = [x for x in token_list if (x in kaku_dict[train_x[i][3]])]
    if(len(candidate) == 0):
        print(f'{i}番目  {train_x[i][3]}→{train_y[i]}')
        print(token_list)
        continue
    if(candidate[0] in train_y[i]):
        counter += 1
        #print(i,'success!!')
print(counter/len(train_x))  #0.746,全ての動詞の原型をfugashiで作成0.69

In [29]:
counter = 0
for i in range(len(train_x)):
    input_ids = tokenizer.encode(f'{train_x[i][1]}{tokenizer.mask_token}{train_x[i][2]}',return_tensors='pt')
    masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    result = model(input_ids)
    index_list = kakuid_dict[tokenizer.encode(train_x[i][3],add_special_tokens=False)[0]]
    candidate_set = [[result.logits[:,masked_index][0][i].item(),tokenizer.decode(i).replace(' ','')] for i in index_list]
    candidate_set.sort(reverse=True)    
    if(candidate_set[0][1] in train_y[i]):
        counter += 1
    else:
        print(f'{i}番目  {train_x[i][3]}→{train_y[i]}\n {candidate_set}')
        print(f'')
print(counter/len(train_x))  #0.918,全ての動詞の原型をfugashiで作成0.71,baselline 86

3番目  に→に
 [[11.224830627441406, 'が'], [9.4724760055542, 'に']]

9番目  から→が
 [[9.567054748535156, 'から'], [7.930869102478027, 'が']]

12番目  で→で
 [[10.197253227233887, 'が'], [8.300032615661621, 'で']]

30番目  と→に
 [[8.223952293395996, 'と']]

54番目  に→が
 [[7.913789749145508, 'に'], [7.6612548828125, 'が']]

57番目  で→が
 [[12.934958457946777, 'で'], [8.343730926513672, 'が']]

64番目  に→が
 [[10.67072582244873, 'に'], [8.125011444091797, 'が']]

65番目  に→が
 [[9.064349174499512, 'に'], [8.697200775146484, 'が']]

66番目  に→に
 [[12.852011680603027, 'が'], [12.328313827514648, 'に']]

73番目  に→が
 [[8.895516395568848, 'に'], [6.967560768127441, 'が']]

102番目  に→が
 [[12.22077465057373, 'に'], [8.891131401062012, 'が']]

110番目  で→で
 [[11.85815143585205, 'が'], [10.285717964172363, 'で']]

141番目  で→で
 [[7.527576446533203, 'が'], [7.199302673339844, 'で']]

142番目  に→が
 [[11.812575340270996, 'に'], [9.898749351501465, 'が']]

145番目  に→が
 [[11.468838691711426, 'に'], [11.042323112487793, 'が']]

147番目  に→が
 [[14.58155345916748, 'に'], [1